In [10]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.2 MB/s eta 0:00:00


In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
tf.__version__

'2.15.0'

In [3]:
fashion_mnist = keras.datasets.fashion_mnist

In [4]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-11-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [12]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [13]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 43s]
val_accuracy: 0.10083333402872086

Best val_accuracy So Far: 0.9175000190734863
Total elapsed time: 00h 03m 13s


In [14]:
model=tuner_search.get_best_models(num_models=1)[0]

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 128)       1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        73792     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 128)               4718720   
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 4795082 (18.29 MB)
Trainable params: 4795082 (18.29 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 14s 7ms/step - loss: 0.1100 - accuracy: 0.9600 - val_loss: 0.2605 - val_accuracy: 0.9157
Epoch 5/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0706 - accuracy: 0.9734 - val_loss: 0.3560 - val_accuracy: 0.9108
Epoch 6/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0469 - accuracy: 0.9834 - val_loss: 0.3697 - val_accuracy: 0.9173
Epoch 7/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0341 - accuracy: 0.9876 - val_loss: 0.4205 - val_accuracy: 0.9142
Epoch 8/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0275 - accuracy: 0.9906 - val_loss: 0.4633 - val_accuracy: 0.9107
Epoch 9/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0204 - accuracy: 0.9932 - val_loss: 0.4908 - val_accuracy: 0.9115
Epoch 10/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0208 - accuracy: 0.9929 - val_loss: 0.5873 - val_a